In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
import matplotlib.pyplot as plt
imageSize = 256
batchSize = 16
channels = 3
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/workspace/DEEP-LEARNING-WITH-TENSORFLOW/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
    shuffle = True,
    image_size = (imageSize, imageSize),
    batch_size = batchSize
)
className = dataset.class_names
for cln in className:
    print(cln)

2024-05-30 04:44:03.486569: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 04:44:03.490734: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 04:44:03.542529: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 04:44:04.846815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 70295 files belonging to 38 classes.
Apple___Apple_scab
Apple___Black_rot
Apple___Cedar_apple_rust
Apple___healthy
Blueberry___healthy
Cherry_(including_sour)___Powdery_mildew
Cherry_(including_sour)___healthy
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Corn_(maize)___Common_rust_
Corn_(maize)___Northern_Leaf_Blight
Corn_(maize)___healthy
Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Grape___healthy
Orange___Haunglongbing_(Citrus_greening)
Peach___Bacterial_spot
Peach___healthy
Pepper,_bell___Bacterial_spot
Pepper,_bell___healthy
Potato___Early_blight
Potato___Late_blight
Potato___healthy
Raspberry___healthy
Soybean___healthy
Squash___Powdery_mildew
Strawberry___Leaf_scorch
Strawberry___healthy
Tomato___Bacterial_spot
Tomato___Early_blight
Tomato___Late_blight
Tomato___Leaf_Mold
Tomato___Septoria_leaf_spot
Tomato___Spider_mites Two-spotted_spider_mite
Tomato___Target_Spot
Tomato___Tomato_Yellow_Leaf_Curl_Virus
Tomato___Tomato_mos

In [2]:
def get_dataset_partitons_tf(dataset, trainingDataset_percent = 0.8, validationDataset_percent = 0.1, testingDataset_percent = 0.1, shuffle = True, shuffle_size = 10000):
    assert (trainingDataset_percent + testingDataset_percent + validationDataset_percent) == 1
    
    dataset_size = len(dataset)
    
    if shuffle:
        dataset = dataset.shuffle(shuffle_size, seed=12)
    
    trainingDataset_size = int(trainingDataset_percent * dataset_size)
    validationDataset_size = int(validationDataset_percent * dataset_size)
    
    train_dataset = dataset.take(trainingDataset_size)
    validation_dataset = dataset.skip(trainingDataset_size).take(validationDataset_size)
    test_dataset = dataset.skip(trainingDataset_size).skip(validationDataset_size)
    
    return train_dataset, validation_dataset, test_dataset

In [3]:
import keras
train_dataset, validation_dataset, test_dataset = get_dataset_partitons_tf(dataset)
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

input_shape = (batchSize, imageSize, imageSize, channels)
n_classes = 38

resize_and_rescale = keras.Sequential([
    keras.layers.Resizing(imageSize, imageSize),
    keras.layers.Rescaling(1.0/255)
])

data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomContrast(0.2),
    keras.layers.RandomTranslation(0.2, 0.2),
    keras.layers.GaussianNoise(0.1),
])

model = keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=input_shape),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(n_classes, activation="softmax")
])

model.build(input_shape=input_shape)
# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)



/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
history = model.fit(
    train_dataset,
    batch_size=batchSize,
    validation_data=validation_dataset,
    verbose=1,
    epochs=50,
)

Epoch 1/50


2024-05-30 04:44:24.646899: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 1273 of 10000
2024-05-30 04:44:44.646128: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 1883 of 10000
2024-05-30 04:44:54.647194: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 2170 of 10000
2024-05-30 04:45:04.654373: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 2455 of 10000
2024-05-30 04:45:24.668615: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 2984 of 10000
2024-05-30 04:45:34.683060: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may tak

3515/3515 ━━━━━━━━━━━━━━━━━━━━ 0s 878ms/step - accuracy: 0.1015 - loss: 3.3061

2024-05-30 05:37:56.036197: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 474 of 10000
2024-05-30 05:38:06.036939: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 1159 of 10000
2024-05-30 05:38:26.032216: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 1811 of 10000
2024-05-30 05:38:36.124014: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 2072 of 10000
2024-05-30 05:38:56.043409: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take a while): 2541 of 10000
2024-05-30 05:39:16.129374: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:7: Filling up shuffle buffer (this may take

3515/3515 ━━━━━━━━━━━━━━━━━━━━ 3446s 945ms/step - accuracy: 0.1015 - loss: 3.3060 - val_accuracy: 0.3437 - val_loss: 2.2098
Epoch 2/50
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 2206s 628ms/step - accuracy: 0.4115 - loss: 1.9950 - val_accuracy: 0.6133 - val_loss: 1.2663
Epoch 3/50
3044/3515 ━━━━━━━━━━━━━━━━━━━━ 4:11 535ms/step - accuracy: 0.6204 - loss: 1.2201

In [ ]:
model.save("model.h5")